In [1]:
!ls

DeepSet FSPool.ipynb DeepSet Sum.ipynb
DeepSet Max.ipynb    data


In [2]:
import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [3]:
188*0.8

150.4

In [4]:
#torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


In [5]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2504], x=[1138, 7], edge_attr=[2504, 4], y=[64], batch=[1138], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2424], x=[1102, 7], edge_attr=[2424, 4], y=[64], batch=[1102], ptr=[65])

Step 3:
Number of graphs in the current batch: 22
DataBatch(edge_index=[2, 1020], x=[452, 7], edge_attr=[1020, 4], y=[22], batch=[452], ptr=[23])



In [6]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_sort_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        self.salidaConv = torch.zeros(0)
        self.salidaPooling = torch.zeros(0)
    def forward(self, x, edge_index, batch):
        
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        self.salidaConv = x
        #print("Primera shape",x.shape)
        # 2. Readout layer
        #print(x.shape)
        x = global_max_pool(x, batch)  # [batch_size, hidden_channels]
        self.salidaPooling = x
        #print(x)
        #x = GlobalPooling(x,batch)
        #x = x.sum(dim=0)
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.lin(x)
        #print(x.shape)
        return x

model = GCN(hidden_channels=18)
print(model)

GCN(
  (conv1): GCNConv(7, 18)
  (conv2): GCNConv(18, 18)
  (conv3): GCNConv(18, 18)
  (lin): Linear(in_features=18, out_features=2, bias=True)
)


In [7]:
model = GCN(hidden_channels=18)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        #print("hola",out)
        #print(data.y)
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.
resultados = []
for i in range(10):
    dataset = dataset.shuffle()

    train_dataset = dataset[:150]
    test_dataset = dataset[150:]

    train_loader = DataLoader(train_dataset.shuffle(), batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset.shuffle(), batch_size=64, shuffle=False)
    model = GCN(hidden_channels=18)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, 50):
        train()
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        #print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    print(f'RUN:{i},Test Acc:{test_acc:.4f}')
    resultados.append(test_acc)

RUN:0,Test Acc:0.6579
RUN:1,Test Acc:0.7632
RUN:2,Test Acc:0.8421
RUN:3,Test Acc:0.7632
RUN:4,Test Acc:0.7895
RUN:5,Test Acc:0.7368
RUN:6,Test Acc:0.7105
RUN:7,Test Acc:0.7368
RUN:8,Test Acc:0.8158
RUN:9,Test Acc:0.7632


In [8]:
res= torch.Tensor(resultados)
print(torch.mean(res))
print(torch.std(res))


tensor(0.7579)
tensor(0.0523)


In [9]:
test_loader = DataLoader(test_dataset[1:], batch_size=1, shuffle=False)
sample = next(iter(test_loader))
sample.batch =torch.tensor(1)
print(sample)
print(sample.y)
out = model(sample.x,sample.edge_index, sample.batch)  # Perform a single forward pass.
pred = out.argmax(dim=1)  # Use the class with highest probability.
print(pred)
correct = 0
correct += int((pred[0] == sample[0].y).sum())  # Check against ground-truth labels.
print("Accuracy: ",(correct/1)*100)#Batch_size

DataBatch(edge_index=[2, 44], x=[21, 7], edge_attr=[44, 4], y=[1], batch=1, ptr=[2])
tensor([1])
tensor([1, 1])
Accuracy:  100.0


In [10]:
#Nodos criticos
Conv_out= model.salidaConv
print(model.salidaConv.shape)
critical_nodes = Conv_out.argmax(0).unique().tolist()
Pool_out = model.salidaPooling
print(Conv_out.shape)
print(Pool_out.shape)
print(critical_nodes)
#print(Conv_out)
grafo_init = sample
A = to_networkx(grafo_init, to_undirected=True)
deg_centrality = list(nx.betweenness_centrality(A, normalized = True, endpoints = False).values())
print(torch.Tensor(deg_centrality).argsort(0).tolist())

torch.Size([21, 18])
torch.Size([21, 18])
torch.Size([2, 18])
[1, 6, 10, 13, 15, 16]


NameError: name 'to_networkx' is not defined

In [ ]:
import networkx as nx
from torch_geometric.utils import *
#loader =  DataLoader(dataset, batch_size=1, shuffle=False)
grafo_init = sample
print(grafo_init)
A = to_networkx(grafo_init, to_undirected=True)
l=[]
for a in A.nodes:
    l.append(a)
ed= []
for e in A.edges:
    ed.append(e)
import igraph as ig
import chart_studio.plotly
Edges= ed
G=ig.Graph(Edges, directed=False)
labels= l
#groups = A.node_attr_dict_factory
N = len(A.nodes)
#print(N)
layt=G.layout('kk', dim=3)
#print(layt)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
#print(Edges)
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]
nx.draw(A)


node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(A.nodes()):
    #print(adjacencies,critical_nodes)
    if adjacencies in critical_nodes :
        node_adjacencies.append('rgb(256,0,0)')
    else:
        node_adjacencies.append('rgb(0,0,0)')
    #node_text.append('# of connections: '+str(len(adjacencies[1])))

In [ ]:
from chart_studio import plotly
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             #color='#ff7f0e',
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )
trace2.marker.color = node_adjacencies
#print(trace2)
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="3D graph visualization of Mutag dataset  by : Ahmed",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',)

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import plotly.graph_objs as go
from  plotly.offline import plot
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected='true')
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

iplot(fig, filename='Les-Miserables')

In [ ]:
import torch
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='PROTEINS')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
#torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:890]
test_dataset = dataset[890:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

In [ ]:
1113*0.8

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_sort_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        self.salidaConv = torch.zeros(0)
        self.salidaPooling = torch.zeros(0)
    def forward(self, x, edge_index, batch):
        
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        self.salidaConv = x
        #print("Primera shape",x.shape)
        # 2. Readout layer
        #print(x.shape)
        x = global_max_pool(x, batch)  # [batch_size, hidden_channels]
        self.salidaPooling = x
        #print(x)
        #x = GlobalPooling(x,batch)
        #x = x.sum(dim=0)
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.lin(x)
        #print(x.shape)
        return x

model = GCN(hidden_channels=18)
print(model)

In [ ]:
model = GCN(hidden_channels=20)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        #print("hola",out)
        #print(data.y)
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.
resultados = []
for i in range(10):
    dataset = dataset.shuffle()

    train_dataset = dataset[:150]
    test_dataset = dataset[150:]

    train_loader = DataLoader(train_dataset.shuffle(), batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset.shuffle(), batch_size=64, shuffle=False)
    model = GCN(hidden_channels=18)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, 50):
        train()
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        #print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    print(f'RUN:{i},Test Acc:{test_acc:.4f}')
    resultados.append(test_acc)

In [ ]:
res= torch.Tensor(resultados)
print(torch.mean(res))
print(torch.std(res))


In [ ]:
test_loader = DataLoader(test_dataset[4:], batch_size=1, shuffle=False)
sample = next(iter(test_loader))
sample.batch =torch.tensor(1)
print(sample)
print(sample.y)
out = model(sample.x,sample.edge_index, sample.batch)  # Perform a single forward pass.
pred = out.argmax(dim=1)  # Use the class with highest probability.
print(pred)
correct = 0
correct += int((pred[0] == sample[0].y).sum())  # Check against ground-truth labels.
print("Accuracy: ",(correct/1)*100)#Batch_size

In [ ]:
import networkx as nx
from torch_geometric.utils import *
#loader =  DataLoader(dataset, batch_size=1, shuffle=False)
grafo_init = sample
print(grafo_init)
A = to_networkx(grafo_init, to_undirected=True)
l=[]
for a in A.nodes:
    l.append(a)
ed= []
for e in A.edges:
    ed.append(e)
import igraph as ig
import chart_studio.plotly
Edges= ed
G=ig.Graph(Edges, directed=False)
labels= l
#groups = A.node_attr_dict_factory
N = len(A.nodes)
print(N)
layt=G.layout('kk', dim=3)
print(layt)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
print(Edges)
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]
nx.draw(A)
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(A.nodes()):
    #print(adjacencies,critical_nodes)
    if adjacencies in critical_nodes :
        node_adjacencies.append('rgb(256,0,0)')
    else:
        node_adjacencies.append('rgb(0,0,0)')
    #node_text.append('# of connections: '+str(len(adjacencies[1])))

In [ ]:
from chart_studio import plotly
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             #color='#ff7f0e',
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )
trace2.marker.color = node_adjacencies
#print(trace2)
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="3D graph visualization of Mutag dataset  by : Ahmed",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="Data source: <a href='https://plotly.com/python/v3/3d-network-graph/</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import plotly.graph_objs as go
from  plotly.offline import plot
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
init_notebook_mode(connected='true')
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

iplot(fig, filename='Les-Miserables')